# Profiles of T/s anomalies

### input files

In [1]:


#time domain
yrst = 2006 ; yren  = 2025
clst = yrst ; clen = 2018


# # EN4
# src  = 'EN4'
# diri = '/g/data/gv90/wrh581/'
# prefix = src+'/EN.4.2.2.?.analysis.l09.'
# sNam = 'salinity' ; tNam = 'temperature'   #variable names


# ORAS5
src  = 'ORAS5'
diri = '/g/data/gv90/wrh581/ORAS5/'  #'/g/data/lo70/hs5546/ORAS5/*/'
sNam = 'vosaline' ; tNam = 'votemper'   #variable names

#output directory
diro = 'gd/Obs-seaice-analysis/plots/'

### time/space sector definitions

In [2]:



#depth
zmin = 0. ; zmax = 400.


#sectors
sector = {
        'Circumpolar' : {'minlon' : 0.,  'maxlon' : 360., 'minlat' : -65., 'maxlat' : -55},
#        'Atl_hemi' : {'minlon' : 300., 'maxlon' : 90.,  'minlat' : -70., 'maxlat' : -60.},
#        'E. Antarctic' : {'minlon' : 30., 'maxlon' : 60.,  'minlat' : -65., 'maxlat' : -62.},
#        'Bellingshausen' : {'minlon' : 277.,  'maxlon' : 295., 'minlat' : -72., 'maxlat' : -65},
        'Weddell' : {'minlon' : 327., 'maxlon' : 10.,  'minlat' : -65., 'maxlat' : -59.},
#        'King Haakon' : {'minlon' : 10., 'maxlon' : 70., 'minlat' : -75., 'maxlat' : -55.}
}




In [3]:
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import numpy as np
import xarray as xr
import cf_xarray as cf
import pandas as pd
import gsw              #TEOS-10 package
import cmocean as cm
import calendar


import logging
logging.getLogger("flox").setLevel(logging.WARNING) #tell flox to get floxed!

import sys
sys.path.append('/g/data/jk72/wrh581/Southern-Ocean-ice-potential')
import zcrit_Hcrit_calc as zcalc
#method of calculating Freeze condition aka SaFT 
SaFT_method = zcalc.freeze_condition_continuous

from dask.distributed import Client
Client=Client(threads_per_worker = 1, memory_limit =0)
Client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 7
Total threads: 7,Total memory: 0 B
Status: running,Using processes: True
Comm: tcp://127.0.0.1:37215,Workers: 7
Dashboard: /proxy/8787/status,Total threads: 7
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:42675,Total threads: 1
Dashboard: /proxy/32781/status,Memory: 0 B
Nanny: tcp://127.0.0.1:43369,


2025-07-18 16:04:50,045 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:44587' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {('concatenate-57739f3788fc69f9ed31c1514b5123d2', 3, 0, 0, 0), ('where-dfc75daea5dffbd94eb958609dc9ddab', 8, 0, 0, 0), ('array-dc7ee2c608f74e52709639e6dc2a5020', 0, 0), ('concatenate-57739f3788fc69f9ed31c1514b5123d2', 8, 0, 0, 0), ('where-0e26ab8cdcd8dbf65a1fec89960f18e4', 8, 0, 0, 0)} (stimulus_id='handle-worker-cleanup-1752818690.0449693')
2025-07-18 16:04:50,096 - distributed.nanny - WARNING - Restarting worker
2025-07-18 16:04:57,128 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:43821' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {('xarray-nav_lat-d166e4f33c3effc437093618d6f6c684', 0), ('where-0790c0d2c7afbd6cb586ff12d3888340', 7, 0, 0, 0), ('where-0e26ab8cdcd8dbf65a1fec89960f18e4', 7, 0, 0, 0), ('where-c687d0564c5870b704dbd3359ff

### Read T/S data

In [4]:


if (src == 'EN4'):

    #get file list for all years and months in range
    yrstring = [str(yr) for yr in  np.arange(yrst,yren+1) ]        #string list of years
    cmd = 'ls '+diri+prefix+'{'+','.join(yrstring)+'}??.nc 2>/dev/null'  #unix command
    fpath = !$cmd


    data = xr.open_mfdataset(fpath, parallel=True).cf.sel(vertical = slice(zmin, zmax)).chunk(-1)

    #add area
    data = data.assign({'wgts' : (data.cf['latitude'].name,   np.cos(np.deg2rad(data.cf['latitude'].values)))}) 

        
    #add some attributes    
    data[sNam].attrs['units'] = 'PSU'
    data[tNam].attrs['units'] = 'K'
    data[tNam].attrs['long_name'] = 'Potential temperature'

elif (src == 'ORAS5'):
    

    def read_ORAS5(var):
        yrstring = [str(yr) for yr in  np.arange(yrst,yren+1) ]        #string list of years
        cmd = 'ls '+diri+var+'/ORAS5_'+var+'_monthly_SOcean_{'+','.join(yrstring)+'}.nc 2>/dev/null'  #unix command
        fpath = !$cmd

        def _ora_preprocess(ds):
            da = ds[var].sel(deptht = slice(zmin,zmax)).isel(x = slice(0,1440))    #bug in ORAS5 data, 'vertical' is not included in the CF dictionary
            return da

          
        return xr.open_mfdataset(fpath,preprocess = _ora_preprocess, chunks = { 'y' :  -1,'x' : -1},  parallel=True)[var]

                  
    data = xr.Dataset({
        tNam : read_ORAS5(tNam),
        sNam : read_ORAS5(sNam)
    })

    #add area weights
    data =  data.assign({'wgts' : (data.cf['latitude'].dims,   np.cos(np.deg2rad(data.cf['latitude'].values)))}) 

    #rename time coord
    data = data.rename({ data.cf['time'].name : 'time'})

    #add correct attribute to temperature var
    data[tNam].attrs['long_name'] = 'Potential temperature'

data

<xarray.Dataset> Size: 32GB
Dimensions:   (deptht: 37, y: 319, x: 1440, time: 234)
Coordinates:
  * deptht    (deptht) float32 148B 0.5058 1.556 2.668 ... 300.9 333.9 370.7
    nav_lat   (y, x) float32 2MB -77.01 -77.01 -77.01 ... -40.98 -40.98 -40.98
    nav_lon   (y, x) float32 2MB 72.75 73.0 73.25 73.5 ... 71.75 72.0 72.25 72.5
  * time      (time) datetime64[ns] 2kB 2006-01-16 2006-02-15 ... 2025-06-16
Dimensions without coordinates: y, x
Data variables:
    votemper  (time, deptht, y, x) float32 16GB dask.array<chunksize=(12, 37, 319, 1440), meta=np.ndarray>
    vosaline  (time, deptht, y, x) float32 16GB dask.array<chunksize=(12, 37, 319, 1440), meta=np.ndarray>
    wgts      (y, x) float32 2MB 0.2248 0.2248 0.2248 ... 0.7549 0.7549 0.7549

### Get sector means

In [5]:
#reset sector def. longitudes (e.g. MOM coords)
pivot = data.cf['longitude'].min().values
if (pivot < 0.):
    pivot = data.cf['longitude'].max().values
    def MOMlon(lon):
        if (lon > pivot): lon = lon - 360.
        return(lon)

    for i in sector:
        xrange              = sector[i]['maxlon'] - sector[i]['minlon']
        #circumpolar special case
        if xrange == 360.:
            sector[i]['minlon'] = -pivot
            sector[i]['maxlon'] = pivot
        else:
            sector[i]['minlon'] = MOMlon(sector[i]['minlon'])
            sector[i]['maxlon'] = MOMlon(sector[i]['maxlon'])



In [6]:

dSiz  = data[tNam].shape    #dimension sizes
dNams = data[tNam].dims           #dimension names
zNam = dNams[1] ; yNam = dNams[2] ; xNam = dNams[3] # make the dim names a bit more 'readable' later



#new dataset to hold sector means (dimensioned time, depth, sector(
nsec = len(sector)
try: 
    sector_mean = data.isel(**{yNam : range(nsec), xNam : 0}).drop_vars(xNam) 
except:
    sector_mean = data.isel(**{yNam : range(nsec), xNam : 0})
sector_mean = sector_mean.rename({ yNam : 'sector'})
sector_mean = sector_mean.assign_coords({'sector' : list(sector.keys())}) #sector names become coords


for sname in sector:
    xmin = sector[sname]['minlon']
    xmax = sector[sname]['maxlon']
    ymin = sector[sname]['minlat']
    ymax = sector[sname]['maxlat']

    #spatial selection (use 'where' in case there's a 2-d lat/lon
    #latitde
    tmp = data.where((data.cf['latitude'] > ymin) & (data.cf['latitude'] <= ymax))
    
    if xmin < xmax:
        tmp = tmp.where((tmp.cf['longitude'] <= xmax) & (tmp.cf['longitude'] > xmin))
    else:
        #this covers case where sector crosses the longitude pivot point
        tmp = tmp.where((tmp.cf['longitude'] <= xmax) | (tmp.cf['longitude'] > xmin))
            
    #weighted mean
    for var in [tNam,sNam]:
        tmp[var] = tmp[var].weighted(tmp.wgts.fillna(0)).mean((yNam, xNam),skipna=True)
        sector_mean[var].loc[{'sector' : sname}] = tmp[var]



sector_mean = sector_mean.chunk(-1)
sector_mean.load()


2025-07-18 16:04:50,108 - distributed.worker - ERROR - failed during get data with tcp://127.0.0.1:39435 -> tcp://127.0.0.1:44587
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/tornado/iostream.py", line 861, in _read_to_buffer
    bytes_read = self.read_from_fd(buf)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/tornado/iostream.py", line 1116, in read_from_fd
    return self.socket.recv_into(buf, len(buf))
ConnectionResetError: [Errno 104] Connection reset by peer

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/distributed/worker.py", line 1783, in get_data
    response = await comm.read(deserializers=serializers)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/distr

KilledWorker: Attempted to run task ('where-c687d0564c5870b704dbd3359ff71bc4', 4, 0, 0, 0) on 4 different workers, but all those workers died while running it. The last worker that attempt to run the task was tcp://127.0.0.1:37993. Inspecting worker logs is often a good next step to diagnose what went wrong. For more information see https://distributed.dask.org/en/stable/killed.html.

### various data transformations/corrections

In [ ]:

#calc P
P = gsw.p_from_z(-sector_mean[zNam], -65.)


#convert Practical to Absolute Salinity if required
if np.isin(sector_mean[sNam].units, ['psu','PSU',1]): 
    for sname in sector:
        sector_mean[sNam].loc[{'sector' : sname}].values = gsw.SA_from_SP(sector_mean[sNam].sel({'sector' : sname}) ,P,
                                       np.mean((sector[sname]['minlon'],sector[sname]['maxlon'] )), 
                                      np.mean((sector[sname]['minlat'],sector[sname]['maxlat']))) 


#convert tenmperature from K to Celsius
if np.isin(sector_mean[tNam].units, ['K','kelvin']):
    sector_mean[tNam].values = sector_mean[tNam].values - 273.15
    sector_mean[tNam].attrs['units'] = 'degC'

#convert potential temperature to conservative temperature
if sector_mean[tNam].long_name == 'Potential temperature':
    sector_mean[tNam].values = gsw.CT_from_pt(sector_mean[sNam].values, sector_mean[tNam].values)


#add local surface freezing temperature (might be useful for plotting
sector_mean = sector_mean.assign(Tf = gsw.CT_freezing(sector_mean[sNam].isel(**{zNam : 0}) , 0., 0.))




### Calculate 'freeze condition' (i.e. where dS/dz can maintain water at freezing temperature), and related critical depths

In [ ]:
# Calc SafT using library function


sector_mean = xr.merge([sector_mean, 
                        SaFT_method(sector_mean[tNam], sector_mean[sNam], 1, sector_mean['Tf'])])


zcrits = zcalc.zcrit_calc_exact(sector_mean['frcon'], zNam)

#sp is seasonal pycnocline (SaFT version of mixed layer depth)
#pp is effectively base of WW/CDW interface (PP = Permanent Pycnocline)
sector_mean = sector_mean.assign({'zcrit_sp' : zcrits[0],  'zcrit_pp' : zcrits[1]}) 

sector_mean.load()

### plot of T, S, SaFT sector_mean

In [ ]:

def sector_anom_plot(plot_sector):

    nrow = 2 ; ncol = 1
    plt.figure(figsize=(20,20), dpi=300)

    selcrit = {'sector' : plot_sector}
    #sector_meanatology (top row)
    colmap = ['',cm.cm.balance, cm.cm.delta]  
    cmax   = ['',0.5, 0.1]  


    
    lcols  = ['magenta','black'] 
    lthck = 3.


    pref = ['','a) T anom','b) S anom']
    cbarlabel = ['','$^o$C','gkg$^{-1}$']



    i = 1
    for var in [tNam, sNam]:
        
        ax = plt.subplot(nrow, ncol, i)

        #subset data
        pdata = sector_mean[var].sel(**selcrit)

        #anomaly
        pdata = pdata.groupby('time.month') - pdata.groupby('time.month').mean ('time')  
        
        cplot = pdata.transpose().plot(ax = ax, vmin = -cmax[i], vmax = cmax[i], cmap = colmap[i],yincrease=False)
        cplot.colorbar.set_label(cbarlabel[i],fontsize=25)
     

        #add critical depth lines
        j = 0
        for line in ['zcrit_sp','zcrit_pp']:
            sector_mean[line].sel(**selcrit).plot(ax=ax, color=lcols[j], linewidth=lthck)
            j = j+1

        #panel title
        plt.title('') ; plt.title(pref[i], loc='left', fontsize = 30)

        #axis labels/tickmarks
        plt.ylabel('depth (m)', fontsize=25) ; plt.xlabel('')
        plt.xticks(fontsize = 20) ; plt.yticks(fontsize=20)
        i = i+1
        
 
     #shared heading
    title = src+' '+plot_sector+' ('+str(sector[plot_sector]['minlon'])+'-'+str(sector[plot_sector]['maxlon'])+'$^o$E ; '+str(sector[plot_sector]['minlat'])+'-'+str(sector[plot_sector]['maxlat'])+'$^o$N )'
    plt.suptitle(title, fontsize=35)


    plt.tight_layout()

In [ ]:
for splot in sector_mean.sector.values:
    sector_anom_plot(splot)
    plt.savefig(diro+src+'_'+splot+'_depth-time_sector_mean_anoms.png')




In [ ]:
Client.close()